In [31]:
import os
import pandas as pd
import numpy as np
from neo4j import GraphDatabase, bearer_auth, RoutingControl
import msal
import logging

## Database connection
Useful read: https://medium.com/neo4j/how-to-integrate-neo4j-with-sso-on-azure-one-login-to-rule-them-all-3492b41a9aa8

In [28]:
DB_URL = "neo4j://localhost:7680"
DB_NAME = "mdrdb20231101v07" # Have to be neo4j for neo4j aura (but keep it, good for testing on local dev env)

In [43]:
CLIENT_ID = "e66aec58-ae0c-4c31-9389-34e0067af3bd"
TENANT_ID = "54e85725-ed2a-49a4-a19e-11c8d29f9a0f"
NEO4J_URI = "neo4j://localhost:7680"
SCOPES = [f"api://{CLIENT_ID}/access-token"]
AZURE_TENANT = f"https://login.microsoftonline.com/{TENANT_ID}"

azure_app = None
neo4j_driver = None
def get_sso_access_token():
    global azure_app
    azure_app = msal.PublicClientApplication(CLIENT_ID, authority=AZURE_TENANT)
    response = azure_app.acquire_token_interactive(scopes=SCOPES, port=51866)
    if not 'access_token' in response:
        logging.error(response)
        raise RuntimeError(response)
    else:
        logging.info("New token acquired")
        return response['id_token']
def get_neo4j_driver():
    global neo4j_driver
    if neo4j_driver is None:
        token = get_sso_access_token()
        neo4j_driver = GraphDatabase.driver(NEO4J_URI, auth=bearer_auth(token))
    return neo4j_driver
     
#get_neo4j_driver().verify_connectivity()
driver = get_neo4j_driver()

ERROR:root:{'error': 'invalid_request', 'error_description': "AADSTS9002327: Tokens issued for the 'Single-Page Application' client-type may only be redeemed via cross-origin requests. Trace ID: 56d72a76-600d-4a0c-b269-6a693d6d7400 Correlation ID: 77cb50b5-1c07-4835-841e-92b9b6a16974 Timestamp: 2024-02-01 16:50:59Z", 'error_codes': [9002327], 'timestamp': '2024-02-01 16:50:59Z', 'trace_id': '56d72a76-600d-4a0c-b269-6a693d6d7400', 'correlation_id': '77cb50b5-1c07-4835-841e-92b9b6a16974'}


RuntimeError: {'error': 'invalid_request', 'error_description': "AADSTS9002327: Tokens issued for the 'Single-Page Application' client-type may only be redeemed via cross-origin requests. Trace ID: 56d72a76-600d-4a0c-b269-6a693d6d7400 Correlation ID: 77cb50b5-1c07-4835-841e-92b9b6a16974 Timestamp: 2024-02-01 16:50:59Z", 'error_codes': [9002327], 'timestamp': '2024-02-01 16:50:59Z', 'trace_id': '56d72a76-600d-4a0c-b269-6a693d6d7400', 'correlation_id': '77cb50b5-1c07-4835-841e-92b9b6a16974'}

In [32]:
records, summary, keys = driver.execute_query(
    'match () return count(*)',
    database_=DB_NAME,
    routing_=RoutingControl.READ
)
print(records)

AttributeError: 'function' object has no attribute 'execute_query'